In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/anime-dataset-2022/Anime.csv


In [2]:
from sklearn.feature_extraction.text import CountVectorizer,HashingVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import seaborn as sns

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
data=pd.read_csv("/kaggle/input/anime-dataset-2022/Anime.csv")

In [4]:
data.shape

(18495, 17)

In [5]:
data.drop(labels=["Release_season","Studio","Japanese_name","Related_anime","staff","Voice_actors","Release_season","Related_Mange","End_year"],axis=1,inplace=True)

In [6]:
data['Content_Warning'].fillna("none",inplace=True)


In [7]:
data["Episodes"][0]=11

/tmp/ipykernel_32/1937055135.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Episodes"][0]=11


In [8]:
data.isna().sum()

Rank                  0
Name                  0
Type                  0
Episodes           8993
Tags                400
Rating             3131
Release_year        383
Description           4
Content_Warning       0
dtype: int64

In [9]:
data.dropna(inplace=True)

In [10]:
data.isna().sum()

Rank               0
Name               0
Type               0
Episodes           0
Tags               0
Rating             0
Release_year       0
Description        0
Content_Warning    0
dtype: int64

In [11]:
data.rename(columns={"Tags":"genre"},inplace=True)

In [12]:
def lower(string):
    data[string]=data[string].apply(lambda x:x.lower())
lower("genre")
lower("Name")
lower("Description")
lower("Content_Warning")
data["Type"]=data["Type"].apply(lambda x:x.upper())

In [13]:
def remove(string):
    return string.replace(" ", "")
data["Type"]=data["Type"].apply(lambda x:remove(x))

In [14]:
data["Name"]=data["Name"].apply(lambda x:x.strip())
data["Description"]=data["Description"].apply(lambda x:x.strip())
data["genre"]=data["genre"].apply(lambda x:x.strip())
data["Content_Warning"]=data["Content_Warning"].apply(lambda x:x.strip())

In [15]:
data["Content_Warning"]=data["Content_Warning"].apply(lambda x:x.replace(",,",","))

In [16]:
cv = CountVectorizer(tokenizer=lambda x:x.split(', ')) # text processing agar memisahkan nama genre yang tepat
by_warning = cv.fit_transform(data['Content_Warning'])

/opt/conda/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [17]:
cos_score_animew = cosine_similarity(by_warning)
pd.DataFrame(cos_score_animew)

,0,1,2,3,4,5,6,7,8,9,...,8216,8217,8218,8219,8220,8221,8222,8223,8224,8225
0,1.000000,0.000000,0.0,0.000000,0.707107,1.000000,0.500000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,1.000000,0.0,0.0,1.000000,0.0,0.0
1,0.000000,1.000000,0.0,0.612372,0.000000,0.000000,0.612372,0.0,0.0,0.408248,...,0.0,0.0,0.0,0.408248,0.000000,0.0,0.0,0.000000,0.0,0.0
2,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,1.0,1.0,0.000000,...,1.0,1.0,1.0,0.000000,0.000000,1.0,1.0,0.000000,1.0,1.0
3,0.000000,0.612372,0.0,1.000000,0.000000,0.000000,0.250000,0.0,0.0,0.500000,...,0.0,0.0,0.0,0.500000,0.000000,0.0,0.0,0.000000,0.0,0.0
4,0.707107,0.000000,0.0,0.000000,1.000000,0.707107,0.353553,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.707107,0.0,0.0,0.707107,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8221,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,1.0,1.0,0.000000,...,1.0,1.0,1.0,0.000000,0.000000,1.0,1.0,0.000000,1.0,1.0
8222,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,1.0,1.0,0.000000,...,1.0,1.0,1.0,0.000000,0.000000,1.0,1.0,0.000000,1.0,1.0
8223,1.000000,0.000000,0.0,0.000000,0.707107,1.000000,0.500000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,1.000000,0.0,0.0,1.000000,0.0,0.0
8224,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,1.0,1.0,0.000000,...,1.0,1.0,1.0,0.000000,0.000000,1.0,1.0,0.000000,1.0,1.0


In [18]:
print(len(cv.get_feature_names_out()))
print(cv.get_feature_names_out())

19
['animal abuse' 'bullying' 'cannibalism' 'domestic abuse' 'drug use'
 'emotional abuse' 'explicit sex' 'explicit violence' 'incest'
 'mature themes' 'none' 'nudity' 'physical abuse' 'prostitution'
 'self-harm' 'sexual abuse' 'sexual content' 'suicide' 'violence']


In [19]:
cv = CountVectorizer(tokenizer=lambda x:x.split(', '))
by_genre= cv.fit_transform(data['genre'])
cos_score_animeg = cosine_similarity(by_genre)

/opt/conda/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [20]:
pd.DataFrame(cos_score_animeg)

,0,1,2,3,4,5,6,7,8,9,...,8216,8217,8218,8219,8220,8221,8222,8223,8224,8225
0,1.000000,0.210042,0.117851,0.447214,0.416667,0.365148,0.320256,0.204124,0.218218,0.819892,...,0.129099,0.129099,0.000000,0.000000,0.117851,0.000000,0.218218,0.218218,0.117851,0.0
1,0.210042,1.000000,0.099015,0.375735,0.140028,0.153393,0.336336,0.085749,0.091670,0.250490,...,0.108465,0.000000,0.000000,0.000000,0.000000,0.099015,0.091670,0.000000,0.099015,0.0
2,0.117851,0.099015,1.000000,0.105409,0.117851,0.000000,0.113228,0.000000,0.000000,0.105409,...,0.182574,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
3,0.447214,0.375735,0.105409,1.000000,0.372678,0.244949,0.501280,0.182574,0.195180,0.533333,...,0.115470,0.115470,0.105409,0.000000,0.000000,0.000000,0.195180,0.195180,0.210819,0.0
4,0.416667,0.140028,0.117851,0.372678,1.000000,0.456435,0.560449,0.204124,0.218218,0.298142,...,0.129099,0.129099,0.000000,0.102062,0.235702,0.000000,0.218218,0.327327,0.117851,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8221,0.000000,0.099015,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.433013,0.333333,1.000000,0.000000,0.154303,0.166667,0.0
8222,0.218218,0.091670,0.000000,0.195180,0.218218,0.239046,0.209657,0.133631,0.142857,0.195180,...,0.169031,0.338062,0.000000,0.133631,0.154303,0.000000,1.000000,0.285714,0.308607,0.0
8223,0.218218,0.000000,0.000000,0.195180,0.327327,0.239046,0.209657,0.000000,0.000000,0.097590,...,0.000000,0.169031,0.000000,0.133631,0.308607,0.154303,0.285714,1.000000,0.617213,0.0
8224,0.117851,0.099015,0.000000,0.210819,0.117851,0.129099,0.226455,0.000000,0.000000,0.210819,...,0.000000,0.182574,0.000000,0.144338,0.166667,0.166667,0.308607,0.617213,1.000000,0.0


In [21]:
anime1 = data.reset_index()
anime1

,index,Rank,Name,Type,Episodes,genre,Rating,Release_year,Description,Content_Warning
0,0,1,demon slayer: kimetsu no yaiba - entertainment...,TV,11.0,"action, adventure, fantasy, shounen, demons, h...",4.60,2021.0,'tanjiro and his friends accompany the hashira...,explicit violence
1,1,2,fruits basket the final season,TV,13.0,"drama, fantasy, romance, shoujo, animal transf...",4.60,2021.0,'the final arc of fruits basket.',"emotional abuse, mature themes, physical abuse..."
2,2,3,mo dao zu shi 3,WEB,12.0,"fantasy, ancient china, chinese animation, cul...",4.58,2021.0,'the third season of mo dao zu shi.',none
3,3,4,fullmetal alchemist: brotherhood,TV,64.0,"action, adventure, drama, fantasy, mystery, sh...",4.58,2009.0,"""the foundation of alchemy is based on the law...","animal abuse, mature themes, violence, domesti..."
4,4,5,attack on titan 3rd season: part ii,TV,10.0,"action, fantasy, horror, shounen, dark fantasy...",4.57,2019.0,'the battle to retake wall maria begins now! w...,"cannibalism, explicit violence"
...,...,...,...,...,...,...,...,...,...,...
8221,15363,15364,shitcom,WEB,1.0,"romance, crude, dark comedy, no dialogue, shor...",1.39,2012.0,a story about love.,none
8222,15364,15365,ex-arm,TV,12.0,"action, sci fi, seinen, androids, robots, base...",1.30,2021.0,"in the year 2014, a high school student who ha...",none
8223,15366,15367,mars of destruction,OVA,1.0,"action, sci fi, aliens, military, shorts, base...",1.11,2005.0,"in the year 2010 the mars exploration vessel, ...",explicit violence
8224,15368,15369,tenkuu danzai skelter heaven,OVA,1.0,"action, drama, mecha, sci fi, shorts, based on...",0.96,2004.0,hovering above the streets of a futuristic tok...,none


In [22]:
anime_liked = input('Apa anime yang anda sukai? ').lower()

index_suka = anime1[anime1['Name']==anime_liked].index
index_suka[0]

Apa anime yang anda sukai?  naruto


469

In [23]:
anime_recom = list(enumerate(cos_score_animeg[index_suka[0]]))
anime_recom[-1]

(8225, 0.0)

In [24]:
your_recom = sorted(anime_recom, key = lambda x: x[1], reverse=True)
your_recom=pd.DataFrame(your_recom)

In [25]:
anime_recom=pd.DataFrame(anime_recom)

In [26]:
your_recom[1][0],your_recom[0][i]

NameError: name 'i' is not defined

In [ ]:
rec=0
for i in range(len(anime_recom)):
    if (anime_recom[1][i])>=0.60:
        print("*",anime1["Name"][i],"--",anime1["genre"][i])
        rec=+1
if rec==0:
    print("can't find any recommendations")

In [ ]:
for i in range(len(your_recom)):
    if (your_recom[1][i])>=0.60:
        print("*",anime1["Name"][your_recom[0][i]],"--",anime1["genre"][your_recom[0][i]])
    

In [ ]:
data.shape

In [ ]:
data["Type"].value_counts()

In [27]:
data.to_excel('animeplanet.xlsx', index=False)